In [49]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ujson
from IPython.display import display, clear_output
import time

In [50]:


# Define the default values
SYMBOL = "BTCUSDT"
CANDLE_PERIOD = "1min"
ORDERBOOK_DATA = f'orderbook_data/{SYMBOL}_{time.strftime("%Y-%m-%d")}.parquet'

In [51]:

def load_and_process_data(file_path, max_retries=5, retry_delay=0.5):
    for attempt in range(max_retries):
        try:
            # Read the Parquet file
            df = pd.read_parquet(file_path)
            
            # Convert timestamp to datetime (if it's not already)
            if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            # Set timestamp as index
            df.set_index('timestamp', inplace=True)
            
            # Parse bids and asks using ujson
            df['bids'] = df['bids'].apply(ujson.loads)
            df['asks'] = df['asks'].apply(ujson.loads)
            
            # Extract the best bid and ask
            df['best_bid'] = df['bids'].apply(lambda x: float(x[0][0]))
            df['best_ask'] = df['asks'].apply(lambda x: float(x[0][0]))
            
            # Calculate the mid price
            df['mid_price'] = (df['best_bid'] + df['best_ask']) / 2
            
            # Resample the data to the specified candle period
            resampled = df['mid_price'].resample(f'{CANDLE_PERIOD}').ohlc()
            
            return df, resampled
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Error reading file: {str(e)}, retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                print(f"Failed to read file after {max_retries} attempts. Error: {str(e)}")
                return None, None

In [52]:
def create_plot(resampled, symbol):
    if resampled is None:
        return go.Figure()  # Return an empty figure if data is None
    
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.03, subplot_titles=(f'{symbol} Candlestick Chart'))
    
    fig.add_trace(go.Candlestick(x=resampled.index,
                                 open=resampled['open'],
                                 high=resampled['high'],
                                 low=resampled['low'],
                                 close=resampled['close'],
                                 name='Candlesticks'))
    
    fig.update_layout(title=f'{symbol} {CANDLE_PERIOD} Candlestick Chart',
                      xaxis_title='Time',
                      yaxis_title='Price (USDT)',
                      xaxis_rangeslider_visible=False)
    
    return fig


In [53]:
def update_plot(symbol=SYMBOL, interval=CANDLE_PERIOD):
    while True:
        df, resampled = load_and_process_data(file_path=ORDERBOOK_DATA)
        if df is not None and resampled is not None:
            fig = create_plot(resampled, symbol)
            
            clear_output(wait=True)
            display(fig)
            
            print(f"Symbol: {symbol}")
            print(f"Interval: {interval}")
            print(f"Number of data points: {len(df)}")
            print(f"Date range: {df.index.min()} to {df.index.max()}")
            print(f"Average mid price: {df['mid_price'].mean():.2f}")
            print(f"Number of candlesticks: {len(resampled)}")
        else:
            print("Failed to update plot due to data reading error.")
        
        # Convert interval to seconds
        if 'min' in interval:
            minutes = float(interval[:-3])
            sleep_time = minutes * 60
        elif 's' in interval:
            sleep_time = float(interval[:-1])
        else:
            raise ValueError("Unsupported interval format. Use 'min' or 's'.")
        
        time.sleep(sleep_time)

In [54]:
# Run the update_plot function to start live updates
update_plot(symbol=SYMBOL, interval=CANDLE_PERIOD)

Error reading file: [Errno 2] No such file or directory: 'orderbook_data/BTCUSDT_2024-07-31.parquet', retrying in 0.5 seconds...
Error reading file: [Errno 2] No such file or directory: 'orderbook_data/BTCUSDT_2024-07-31.parquet', retrying in 0.5 seconds...
Error reading file: [Errno 2] No such file or directory: 'orderbook_data/BTCUSDT_2024-07-31.parquet', retrying in 0.5 seconds...


KeyboardInterrupt: 